In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import corner
import astropy.cosmology as cosmo
from astropy.cosmology import Planck18
import astropy.units as unit

seed = 1023123283

$$
P(\vec{\lambda}|m_{obs},z_{obs}) \propto \frac{\pi(\vec{\lambda})\prod\limits_{i}\int dmdz P(m_{obs},z_{obs}|m,z) P(m,z|\vec{\lambda})}{\beta(\vec{\lambda})^{N_{obs}}}
$$

$$
\propto \frac{\pi(\vec{\lambda})\prod\limits_{i}\int dmdz \frac{P(m,z|m_{obs},z_{obs})}{\pi(m,z)} P(m,z|\vec{\lambda})}{\beta(\vec{\lambda})^{N_{obs}}}
$$

$$
\approx \frac{\pi(\vec{\lambda})\prod\limits_{i}\sum\limits_{m,z \sim P(m,z|m_{obs},z_{obs})} \frac{P(m,z|\vec{\lambda})}{\pi(m,z)}}{\beta(\vec{\lambda})^{N_{obs}}}
$$

In our case $\pi(m,z)=U(m,z)$. So the hyperposterior is:
$$
P(\vec{\lambda}|m_{obs},z_{obs}) \propto \frac{\pi(\vec{\lambda})\prod\limits_{i}\sum\limits_{m,z \sim P(m,z|m_{obs},z_{obs})} P(m,z|\vec{\lambda})}{\beta(\vec{\lambda})^{N_{obs}}}
$$

Selection function $\beta (\vec{\lambda})$ is:

$$
\beta (\vec{\lambda}) = \int_{m,z}dmdz P(det|m,z)P(m,z|\vec{\lambda})
$$

$$
= \int_{m,z}dmdz \frac{P(m,z|det,draw)P(det|draw)}{P(m,z|draw)}P(m,z|\vec{\lambda})
$$

$$
\approx \sum\limits_{m,z \sim P(m,z|det,draw)} \frac{P(m,z|\vec{\lambda})}{{P(m,z|draw)}}
$$

# $m,z \sim P(m,z|det,draw)$

In [2]:
Ninj = 10000000

In [3]:
# mass
m = np.random.uniform(20.0,65.0,Ninj)

# redshift
z = np.random.uniform(0.001,2.0,Ninj)

In [4]:
from scipy.interpolate import RectBivariateSpline, interp1d

In [5]:
with h5py.File('optimal_snr.h5', 'r') as inp:
    mass = np.array(inp['ms'])
    osnrs = np.array(inp['SNR'])
inp.close()

osnr_interp = RectBivariateSpline(mass, mass, osnrs)

def optimal_snr(m, z):
    m1z = m*(1.0+z)
    m2z = m*(1.0+z)
    dl = Planck18.luminosity_distance(z).to(unit.Gpc).value
    return osnr_interp.ev(m1z, m2z)/dl

def rho(optimal_snr, Theta):
    return optimal_snr*Theta

In [6]:
Theta = np.random.beta(2.0, 4.0, Ninj)

In [7]:
rhoo = rho(optimal_snr(m, z), Theta) + np.random.randn(Ninj)

In [8]:
mask = rhoo>8.0
rhoo_det = rhoo[mask]
Ndet = len(rhoo_det)
print(Ndet)

2459341


In [9]:
mdet = m[mask]
zdet = z[mask]

In [10]:
mdet.shape, zdet.shape

((2459341,), (2459341,))

In [11]:
with h5py.File('Selection_samples.h5', 'w') as inp:
    inp.create_dataset('mdet', data=mdet)
    inp.create_dataset('zdet', data=zdet)
inp.close()

$$
P(m,z|draw) = 1
$$